### Se obtinenen las imágenes y se almacenan

In [5]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import time

from raven import Client
client = Client('https://6293821d2e964169aed64c863b9f5eb0@sentry.io/1473077')

In [6]:
estilos = ['realism', 'surrealism', 'abstract-art', 'pop-art']
pags = list(range(1, 11))

In [7]:
def descarga_img(url, carpeta, tipo):
    '''
    descarga las imágenes
    a la carpeta especificada
    tipo : train ,test, validar
    '''
    
    path = 'imagenes/{}/{}/'.format(tipo, carpeta)
    try:
        nombre = url.split('/')[-1]
        imagen = requests.get(url)

        with open(path + nombre, 'wb') as f:  
            f.write(imagen.content)
            
    except Exception as e:
        client.captureException()
        return 'Error: ' + str(e)            

In [8]:
def datos_datos(estilos, pags):
    '''
    obtiene los json y los almacena 
    en una lista
    '''
    lista_json = []
    url = 'https://www.wikiart.org/en/paintings-by-style/'
    try:
        for e in estilos:
            for i in pags:
                url_json = '{}{}?json=2&page={}'.format(url, e, i)
                print(url_json)
                response = requests.get(url_json).json()

                for item in response['Paintings']:
                    item.update( {"estilo": e})

                lista_json.append(response['Paintings'])
                time.sleep(10 / 100)
                
    except Exception as e:
        client.captureException()
        return 'Error: ' + str(e)                
            
    return lista_json
    

In [25]:
lista = datos_datos(estilos, pags)

https://www.wikiart.org/en/paintings-by-style/realism?json=2&page=12
https://www.wikiart.org/en/paintings-by-style/realism?json=2&page=13
https://www.wikiart.org/en/paintings-by-style/surrealism?json=2&page=12
https://www.wikiart.org/en/paintings-by-style/surrealism?json=2&page=13
https://www.wikiart.org/en/paintings-by-style/abstract-art?json=2&page=12
https://www.wikiart.org/en/paintings-by-style/abstract-art?json=2&page=13
https://www.wikiart.org/en/paintings-by-style/pop-art?json=2&page=12
https://www.wikiart.org/en/paintings-by-style/pop-art?json=2&page=13


In [ ]:
def crea_df(lista_json):
    '''
    Devuelve un dataframe compuesto
    por la lista de json
    '''
    df = pd.DataFrame()
    for i in lista:
        df = df.append(i, ignore_index = True)
        
    return df

In [26]:
df = pd.DataFrame()
for i in lista:
    df = df.append(i, ignore_index = True)

In [ ]:
df = crea_df(lista_json)

In [27]:
df.head()

,albums,artistName,artistUrl,estilo,flags,height,id,image,images,map,paintingUrl,title,width,year
0,None,Aleksey Savrasov,/en/aleksey-savrasov,realism,2,750,577276c2edc2cb3880d289a2,https://uploads2.wikiart.org/images/aleksey-sa...,None,0123**67*,/en/aleksey-savrasov/view-of-the-kremlin-from-...,View of the Kremlin from the Krimsky Bridge in...,1000,1851
1,None,Pavel Fedotov,/en/pavel-fedotov,realism,2,2135,57727cb7edc2cb3880e55dff,https://uploads3.wikiart.org/images/pavel-fedo...,None,01234567*,/en/pavel-fedotov/not-detected-270365,"Encore, Encore",3092,1851
2,None,Pavel Fedotov,/en/pavel-fedotov,realism,2,699,57727cb8edc2cb3880e55e15,https://uploads3.wikiart.org/images/pavel-fedo...,None,0123**67*,/en/pavel-fedotov/officer-and-his-orderly-1851,Officer and His Orderly,574,1850-1851
3,None,Pavel Fedotov,/en/pavel-fedotov,realism,2,753,57727cb8edc2cb3880e55f35,https://uploads4.wikiart.org/images/pavel-fedo...,None,0123**67*,/en/pavel-fedotov/portrait-of-olga-ivanovna-an...,Portrait of Olga Ivanovna and Fedor Ivanov,542,1851
4,None,Pavel Fedotov,/en/pavel-fedotov,realism,2,738,57727cb8edc2cb3880e55f45,https://uploads6.wikiart.org/images/pavel-fedo...,None,0123**67*,/en/pavel-fedotov/portrait-of-olga-ivanovna-de...,Portrait of Olga Ivanovna Demonkala,551,1851


In [28]:
df.shape

(480, 14)

In [29]:
df.to_csv('/data/datos-test.csv', index = False)

In [30]:
df.iloc[21]['image']

'https://uploads8.wikiart.org/images/theodore-rousseau/view-of-the-belfry-of-orleans.jpg'

In [31]:
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

albums    480
images    475
dtype: int64

In [32]:
imagenes = df[['estilo', 'image']]

In [33]:
for e in range(0, imagenes.shape[0] ):
    print(imagenes.iloc[e]['image'])
    descarga_img(imagenes.iloc[e]['image'], imagenes.iloc[e]['estilo'], 'validar')
    time.sleep(5 / 1000)

https://uploads2.wikiart.org/images/aleksey-savrasov/view-of-the-kremlin-from-the-krimsky-bridge-in-inclement-weather-1851.jpg
https://uploads3.wikiart.org/images/pavel-fedotov/not-detected-270365.jpg
https://uploads3.wikiart.org/images/pavel-fedotov/officer-and-his-orderly-1851.jpg
https://uploads4.wikiart.org/images/pavel-fedotov/portrait-of-olga-ivanovna-and-fedor-ivanov-1851.jpg
https://uploads6.wikiart.org/images/pavel-fedotov/portrait-of-olga-ivanovna-demonkala-1851.jpg
https://uploads2.wikiart.org/images/pavel-fedotov/portrait-of-s-s-krylov-1851.jpg
https://uploads6.wikiart.org/images/pavel-fedotov/wintry-street-on-vasilievsky-island-st-petersburg.jpg
https://uploads0.wikiart.org/images/pavel-fedotov/the-major-s-marriage-proposal-1851.jpg
https://uploads8.wikiart.org/images/ernest-meissonier/dimanche-poissy-1851.jpg
https://uploads3.wikiart.org/images/rosa-bonheur/going-to-market-1851.jpg
https://uploads3.wikiart.org/00123/images/jean-francois-millet/the-sower-1851.jpg
https://u

https://uploads7.wikiart.org/images/aleksey-savrasov/gulf-of-finland-1854.jpg
https://uploads2.wikiart.org/images/aleksey-savrasov/landscape-with-pine-1854.jpg
https://uploads0.wikiart.org/images/aleksey-savrasov/old-pine-1854.jpg
https://uploads5.wikiart.org/images/aleksey-savrasov/pine-1854.jpg
https://uploads5.wikiart.org/images/aleksey-savrasov/pines-adjacent-to-the-root-of-the-barrel-1854.jpg
https://uploads2.wikiart.org/images/aleksey-savrasov/sea-shore-in-the-vicinity-oranienbaum-1854.jpg
https://uploads7.wikiart.org/images/aleksey-savrasov/stumps-1854.jpg
https://uploads7.wikiart.org/images/aleksey-savrasov/tropical-plants-1854.jpg
https://uploads5.wikiart.org/images/aleksey-savrasov/view-in-the-neighbourhood-of-oranienbaum-1854.jpg
https://uploads7.wikiart.org/images/eugene-boudin/le-havre-francais-tower.jpg
https://uploads3.wikiart.org/images/nikolai-ge/portrait-of-nikolay-o-ge-artist-s-father.jpg
https://uploads7.wikiart.org/images/johan-hendrik-weissenbruch/at-elshout-sun-1

https://uploads7.wikiart.org/images/salvador-dali/gala-and-the-angelus-of-millet-before-the-imminent-arrival-of-the-conical-anamorphoses.jpg
https://uploads0.wikiart.org/images/salvador-dali/geological-evolution-1933.jpg
https://uploads5.wikiart.org/images/salvador-dali/myself-at-the-age-of-ten-when-i-was-the-grasshopper-child.jpg
https://uploads1.wikiart.org/images/salvador-dali/necrophiliac-fountain-flowing-from-a-grand-piano-1933.jpg
https://uploads5.wikiart.org/images/salvador-dali/portrait-of-gala-1933.jpg
https://uploads0.wikiart.org/images/salvador-dali/portrait-of-gala-with-two-lamb-chops-balanced-on-her-shoulder.jpg
https://uploads4.wikiart.org/images/salvador-dali/retrospective-bust-of-a-woman.jpg
https://uploads5.wikiart.org/images/salvador-dali/sugar-sphinx.jpg
https://uploads1.wikiart.org/images/salvador-dali/the-architectonic-angelus-of-millet.jpg
https://uploads1.wikiart.org/images/salvador-dali/the-enigma-of-william-tell.jpg
https://uploads6.wikiart.org/images/salvador-

https://uploads2.wikiart.org/images/perle-fine/ideomorphic-composition-1-1942.jpg
https://uploads2.wikiart.org/images/richard-pousette-dart/symphony-no-1-the-transcendental-1942.jpg
https://uploads0.wikiart.org/images/lawren-harris/abstract-painting-20-1942.jpg
https://uploads4.wikiart.org/images/lawren-harris/lsh-21-1942.jpg
https://uploads7.wikiart.org/images/auguste-herbin/composition-sur-le-mots-poires-pommes-p-ches-1942.jpg
https://uploads0.wikiart.org/images/willi-baumeister/africa-i-1942.jpg
https://uploads0.wikiart.org/images/willi-baumeister/jurassic-1942.jpg
https://uploads4.wikiart.org/images/willi-baumeister/perforation-on-yellow-1942.jpg
https://uploads0.wikiart.org/images/louis-schanker/wall-handball-study-1942.jpg
https://uploads3.wikiart.org/00180/images/maria-helena-vieira-da-silva/vieira-da-silva-le-desastre-ou-la-guerre-1942.jpg
https://uploads4.wikiart.org/images/wassily-kandinsky/conglomerat-1943.jpg
https://uploads1.wikiart.org/images/wassily-kandinsky/twilight-19

https://uploads7.wikiart.org/images/richard-hamilton/i-m-dreaming-of-a-black-christmas-1971.jpg
https://uploads7.wikiart.org/images/saul-steinberg/untitled-1971.jpg
https://uploads6.wikiart.org/images/audrey-flack/macarena-esperanza-1971.jpg
https://uploads8.wikiart.org/images/audrey-flack/macarena-of-miracles-1971.jpg
https://uploads0.wikiart.org/images/patrick-caulfield/interior-evening-1971.jpg
https://uploads0.wikiart.org/images/patrick-caulfield/interior-morning-1971.jpg
https://uploads6.wikiart.org/images/patrick-caulfield/interior-noon-1971.jpg
https://uploads7.wikiart.org/images/patrick-caulfield/interior-night-1971.jpg
https://uploads8.wikiart.org/images/jim-dine/picabia-i-cheer-1971.jpg
https://uploads1.wikiart.org/images/jim-dine/picabia-ii-forgot-1971.jpg
https://uploads2.wikiart.org/images/jim-dine/picabia-iii-groans-1971.jpg
https://uploads5.wikiart.org/images/evelyne-axell/la-terre-est-ronde-variation-sur-le-paysage-1971.jpg
https://uploads2.wikiart.org/images/evelyne-ax